# Bedrock with LangChain using a Prompt that includes Context

## Introduction

In this notebook we show you how to generate an email response to a customer who was not happy with the quality of customer service that they received from the customer support engineer. We will provide additional context to the model by providing the contents of the actual email that was received from the unhappy customer.

Because of additional context in the prompt, the text produced by the Amazon Titan Large language model in this notebook is of much better quality and relevance than the content produced earlier through zero-shot prompts.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example creates a custom LangChain prompt template for adding context to the text generation request. 

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
In the previous example `01_zero_shot_generation.ipynb`, we explored how to use LangChain framework to communicate with Amazon Bedrock API. In this notebook we will try to add a bit more complexity with the help of `PromptTemplates` to leverage the LangChain framework for the similar use case. `PrompTemplates` allow you to create generic shells which can be populated with information later and get model outputs based on different scenarios.

As part of this notebook we will explore the use of Amazon Bedrock integration within LangChain framework and how it could be used to generate text with the help of `PromptTemplate`.

#### Pattern
We will simply provide the LangChain implementation of Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/bedrock_langchain.jpg)

#### Use case
To demonstrate the generation capability of models in Amazon Bedrock, let's take the use case of email generation.

#### Persona
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

#### Implementation
To fulfill this use case, we will show you how to generate an email with a thank you note based on the customer's previous email. We will use the Amazon Titan Text Large model using the Amazon Bedrock LangChain integration. 


## Setup
Before we get started with the implementation we have to make sure that the required boto3 and botocore packages are installed. These will be used to leverage the Amazon Bedrock API client.

In [ ]:
# Make sure you run `download-dependencies.sh` from the root of the repository to download the dependencies before running this cell
%pip install ../dependencies/botocore-1.29.162-py3-none-any.whl ../dependencies/boto3-1.26.162-py3-none-any.whl ../dependencies/awscli-1.27.162-py3-none-any.whl --force-reinstall

Let's begin with creating an instance of Bedrock class from llms. This expects a `model_id` which is the ARN of the model available in Amazon Bedrock. 

Optionally you can pass on a previously created boto3 client as well as some `model_kwargs` which can hold parameters such as `temperature`, `topP`, `maxTokenCount` or `stopSequences` (more on parameters can be explored in Amazon Bedrock console).

Available text generation models under Amazon Bedrock have the following IDs:

- amazon.titan-tg1-large
- ai21.j2-grande-instruct
- ai21.j2-jumbo-instruct
- anthropic.claude-instant-v1
- anthropic.claude-v1

#### ⚠️⚠️⚠️ Execute the following cells before running this notebook ⚠️⚠️⚠️

For a detailed description on what the following cells do refer to [Bedrock boto3 setup](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

For this notebook we also need langchain version >= 0.0.190 which has Amazon Bedrock class implemented under llms module. Also we are installing the transformers framework from HuggingFace, which we will use to quickly count the number of tokens in the input prompt.

In [ ]:
%pip install langchain==0.0.190 --quiet
%pip install transformers==4.24.0 --quiet

In [ ]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [ ]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
boto3_bedrock = bedrock.get_bedrock_client(os.environ.get('BEDROCK_ASSUME_ROLE', None))

## Invoke the Bedrock LLM Model

For more details for the parameters please refer to the Bedrock API page.

In [ ]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v1",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


## Create a LangChain custom prompt template

By creating a template for the prompt we can pass it different input variables to it on every run. This is useful when you have to generate content with different input variables that you may be fetching from a database.

Previously we hardcoded the prompt, it might be the case that you have multiple customers sending similar negative feedback and you now want to use each of those customer's emails and respond to them with an apology but you also want to keep the response a bit personalized. In the following cell we are exploring how you can create a `PromptTemplate` to achieve this pattern.

In [ ]:
from langchain import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""Create an apology email from the Service Manager {customerServiceManager} to {customerName}. 
   in response to the following feedback that was received from the customer: {feedbackFromCustomer}.
   """
    
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

## Invoke again

invoke using the prompt template and expect to see a curated response back

In [ ]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]  

print_ww(email)

## Summary

To conclude we learnt that invoking the LLM without any context might not yield the desired results. By adding context and further using the the prompt template to constrain the output from the LLM we are able to successfully get our desired output